In [4]:
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By

cols = ['Nama', 'Harga Diskon', 'Harga Awal', 'Diskon', 'Terjual', 'Kota/Kab', 'Toko', 'Rating']
df = pd.DataFrame(columns = cols)

service = Service("C:\Program Files\edgedriver_win64")
driver = webdriver.Edge(service = service)

url = 'https://www.tokopedia.com/find/sewa-photobooth'

driver.get(url)
driver.implicitly_wait(5)

In [5]:
def is_float(string):
    try:
        float(string)
        return True
    
    except ValueError:
        return False

In [8]:
list_items

['sewa photobooth',
 'Rp300.000',
 'Jakarta Selatan',
 'Bimasakti Tenda',
 'Paket Sewa Photobooth Polaroid 2 Jam Unlimited JAKARTA & BEKASI',
 'Rp1.799.000',
 'Jakarta Timur',
 'Fotoimoet',
 'Sewa properti photobooth',
 'Rp1.300.000',
 'Jakarta Timur',
 'unicdedecor',
 'Sewa mesin printer photobooth + laptop + operator',
 'Rp650.000',
 'Surabaya',
 'ivory photo video sby',
 'Sewa Photobooth',
 'Rp500.000',
 'Jakarta Barat',
 'Sewa properti photobooth',
 'Rp1.000.000',
 'Jakarta Timur',
 'unicdedecor',
 'sewa photobooth rustic',
 'Rp1.200.000',
 'Jakarta Barat',
 'doorgift citra',
 'Sewa properti photobooth',
 'Rp1.300.000',
 'Jakarta Timur',
 'unicdedecor',
 'Sewa Printer Photobooth Jakarta Termurah',
 'Rp700.000',
 'Jakarta Timur',
 'Kreatif Digital',
 'Box boneka Barbie props dekorasi photobooth pesta backdrop sewa/ beli',
 'Rp1.500.000',
 'Jakarta Utara',
 'Favor Party Supplies',
 'Jasa sewa Photobooth',
 'Rp2.500.000',
 'Jakarta Timur',
 'ArfanProject',
 'sewa dekorasi backdrop lam

In [6]:
df = pd.DataFrame(columns = cols)

res = driver.find_elements(By.XPATH, "//div[@class='css-em6f9j']")

for k in range(len(res)):
    list_items = res[k].text.split('\n')
    list_items = [item for item in list_items if 'Cashback' not in item and 'Ad' not in item]

    i = 0
    while i < len(list_items):
        nama = ''
        harga_diskon = ''
        harga_awal = ''
        diskon = ''
        cashback = ''
        terjual = ''
        kota_kab = ''
        toko = ''
        rating = ''

        count = i
        j = 0
        ada_harga = False
        while (j < 8) and (count + j < len(list_items)):

            if len(list_items[count + j]) >= 15:
                if nama == '':
                    nama = list_items[count + j]

                elif (toko != '') or (nama != '') or (kota_kab != ''):
                    j = 8

            elif list_items[count + j].startswith('Rp'):
                if ada_harga:
                    harga_awal = list_items[count + j]
                    kota_kab = list_items[count + j + 1]
                    toko = list_items[count + j + 2]
                    j += 2

                else:
                    harga_diskon = list_items[count + j]
                    ada_harga = True
                    if not list_items[count + j + 1].startswith('Rp'):
                        kota_kab = list_items[count + j + 1]
                        toko = list_items[count + j + 2]
                        j += 2

            elif (not list_items[count + j].startswith('Cashback')) and (list_items[count + j].endswith('%')):
                diskon = list_items[count + j]

            elif is_float(list_items[count + j]):
                rating = list_items[count + j]

            elif list_items[count + j].endswith('terjual'):
                terjual = list_items[count + j].split(' ')[0]

            j += 1

        if harga_awal == '':
            harga_awal = harga_diskon
            harga_diskon = ''

        temp = [nama, harga_diskon, harga_awal, diskon, terjual, kota_kab, toko, rating]
        i += len([_ for _ in temp if _ != ''])

        df.loc[len(df)] = temp

In [13]:
df

,Nama,Harga Diskon,Harga Awal,Diskon,Terjual,Kota/Kab,Toko,Rating
0,sewa photobooth,,Rp300.000,,,Jakarta Selatan,Bimasakti Tenda,
1,Paket Sewa Photobooth Polaroid 2 Jam Unlimited...,,Rp1.799.000,,,Jakarta Timur,Fotoimoet,
2,Sewa properti photobooth,,Rp1.300.000,,,Jakarta Timur,unicdedecor,
3,Sewa mesin printer photobooth + laptop + operator,,Rp650.000,,,Surabaya,ivory photo video sby,
4,sewa photobooth rustic,,Rp1.200.000,,,Jakarta Barat,doorgift citra,
...,...,...,...,...,...,...,...,...
58,photobooth wedding murah jabodetabek,,Rp1.800.000,,,Jakarta Selatan,wajendra29,5.0
59,photo booth ulang tahun murah,,Rp1.500.000,,,Jakarta Selatan,wajendra29,
60,MobileTrans - Wa Transfer dan Line Transfer Or...,,Rp125.000,,,Kab. Bekasi,NusantaraLisensi,5.0
61,SEWA WIFI JEPANG | JAPAN WIFI | SEWA JAPAN WIFI,,Rp49.000,,,Jakarta Pusat,japantrips,4.9


In [ ]:
temp = df.copy()

In [14]:
df['Harga Awal'] = df['Harga Awal'].apply(lambda x: re.sub(r'[^0-9]', '', x) if type(x) == str else str(x))
df['Harga Diskon'] = df['Harga Diskon'].apply(lambda x: re.sub(r'[^0-9]', '', x) if type(x) == str else str(x))
mask = (df['Harga Diskon'] == '')
df.loc[mask, 'Harga Diskon'] = df.loc[mask, 'Harga Awal']
df = df.dropna(subset = ['Harga Awal', 'Harga Diskon'])
df = df.loc[~((df['Harga Awal'] == '') & (df['Harga Diskon'] == ''))]
df['Harga Awal'] = df['Harga Awal'].astype(float)
df['Harga Diskon'] = df['Harga Diskon'].astype(float)

df['Diskon'] = round((df['Harga Awal'] - df['Harga Diskon']) / df['Harga Awal'] * 100, 1)

df

,Nama,Harga Diskon,Harga Awal,Diskon,Terjual,Kota/Kab,Toko,Rating
0,sewa photobooth,300000.0,300000.0,0.0,,Jakarta Selatan,Bimasakti Tenda,
1,Paket Sewa Photobooth Polaroid 2 Jam Unlimited...,1799000.0,1799000.0,0.0,,Jakarta Timur,Fotoimoet,
2,Sewa properti photobooth,1300000.0,1300000.0,0.0,,Jakarta Timur,unicdedecor,
3,Sewa mesin printer photobooth + laptop + operator,650000.0,650000.0,0.0,,Surabaya,ivory photo video sby,
4,sewa photobooth rustic,1200000.0,1200000.0,0.0,,Jakarta Barat,doorgift citra,
5,Sewa properti photobooth,1300000.0,1300000.0,0.0,,Jakarta Timur,unicdedecor,
6,Sewa Printer Photobooth Jakarta Termurah,700000.0,700000.0,0.0,,Jakarta Timur,Kreatif Digital,
7,Box boneka Barbie props dekorasi photobooth pe...,1500000.0,1500000.0,0.0,,Jakarta Utara,Favor Party Supplies,
8,Jasa sewa Photobooth,2500000.0,2500000.0,0.0,,Jakarta Timur,ArfanProject,
9,sewa dekorasi backdrop lamaran/ akad nikah/ ph...,1200000.0,1200000.0,0.0,,Jakarta Barat,To Amanah,


In [15]:
df.to_csv('photobooth.csv', index = False)